In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import joblib

In [3]:
# Cargar el modelo guardado
modelo = joblib.load("..\prediccion_reforestacion\modelo\modelo_hibrido_0.9269.pkl")

# Cargar los datos originales para obtener todas las estaciones y distritos
df_original = pd.read_csv('..\prediccion_reforestacion\data\data_monitoreo_contaminantes_filtrado_best.csv')

# Datos históricos de PM10
historico_pm10 = {
    2015: 68.51, 2016: 71.85, 2017: 73.25, 2018: 69.36, 
    2019: 56.26, 2020: 53.22, 2021: 60.80, 2022: 56.51,
    2023: 41.89, 2024: 46.72
}

# Datos de arborización SERPAR
arboles_serpar = {
    2020: 82637,
    2021: 160932,
    2022: 150136,
    2023: 86957,
    2024: 115548
}

# Constantes globales
TASA_SUPERVIVENCIA = 0.7
ARBOLES_PROMEDIO = np.mean(list(arboles_serpar.values()))
VOLUMEN_AIRE = 2819000000 * 10  # área de Lima * altura promedio de mezcla
TARGET_PM10_MIN = 27  # Límite inferior del rango objetivo
TARGET_PM10_MAX = 40.5  # Límite superior del rango objetivo

<>:2: SyntaxWarning: invalid escape sequence '\p'
<>:5: SyntaxWarning: invalid escape sequence '\p'
<>:2: SyntaxWarning: invalid escape sequence '\p'
<>:5: SyntaxWarning: invalid escape sequence '\p'
C:\Users\marli\AppData\Local\Temp\ipykernel_21388\836284415.py:2: SyntaxWarning: invalid escape sequence '\p'
  modelo = joblib.load("..\prediccion_reforestacion\modelo\modelo_hibrido_0.9269.pkl")
C:\Users\marli\AppData\Local\Temp\ipykernel_21388\836284415.py:5: SyntaxWarning: invalid escape sequence '\p'
  df_original = pd.read_csv('..\prediccion_reforestacion\data\data_monitoreo_contaminantes_filtrado_best.csv')
c:\Users\marli\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator OneHotEncoder from version 1.2.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-ma

ValueError: node array from the pickle has an incompatible dtype:
- expected: {'names': ['left_child', 'right_child', 'feature', 'threshold', 'impurity', 'n_node_samples', 'weighted_n_node_samples', 'missing_go_to_left'], 'formats': ['<i8', '<i8', '<i8', '<f8', '<f8', '<i8', '<f8', 'u1'], 'offsets': [0, 8, 16, 24, 32, 40, 48, 56], 'itemsize': 64}
- got     : [('left_child', '<i8'), ('right_child', '<i8'), ('feature', '<i8'), ('threshold', '<f8'), ('impurity', '<f8'), ('n_node_samples', '<i8'), ('weighted_n_node_samples', '<f8')]

In [16]:
### 3. Preparar escenarios
def crear_datos_prediccion(start_year=2025, end_year=2035):
    """
    Crea DataFrame con todas las combinaciones necesarias para la predicción
    """
    # Obtener todas las estaciones y distritos únicos
    estaciones = df_original['ESTACION'].unique()
    distritos = df_original['DISTRITO'].unique()
    
    datos = []
    for año in range(start_year, end_year + 1):
        for mes in range(1, 13):
            for estacion, distrito in zip(estaciones, distritos):
                for hora in range(24):
                    dia_semana = pd.Timestamp(year=año, month=mes, day=1).weekday()
                    es_fin_semana = 1 if dia_semana >= 5 else 0
                    datos.append({
                        'ESTACION': estacion,
                        'DISTRITO': distrito,
                        'HORA': hora,
                        'DIA_SEMANA': dia_semana,
                        'MES': mes,
                        'AÑO': año,
                        'ES_FIN_DE_SEMANA': es_fin_semana
                    })
    return pd.DataFrame(datos)

def calcular_arboles_necesarios(df_pred, target_pm10=33.75, start_year=2025):
    """
    Calcula cuántos árboles se necesitan plantar por año para alcanzar el objetivo de PM10
    """
    prediccion_base = modelo.predict(df_pred)
    años_unicos = df_pred['AÑO'].unique()
    
    # Calcular reducción por árbol (5217.4 g/año)
    reduccion_diaria_por_arbol = 5217.4 / 365
    
    arboles_necesarios_por_año = {}
    arboles_acumulados = {}
    pm10_proyectado = {}
    
    for i, año in enumerate(años_unicos):
        mask_año = df_pred['AÑO'] == año
        pm10_base_año = np.mean(prediccion_base[mask_año])
        
        # Calcular cuánta reducción se necesita
        reduccion_necesaria = max(0, pm10_base_año - target_pm10)
        
        # Calcular cuántos árboles se necesitan para esta reducción
        # Considerando los árboles acumulados de años anteriores
        arboles_acumulados_previos = sum(arboles_necesarios_por_año.get(a, 0) * TASA_SUPERVIVENCIA for a in range(start_year, año))
        
        # Calcular reducción ya lograda por árboles previos
        reduccion_ya_lograda = (reduccion_diaria_por_arbol * arboles_acumulados_previos * 1000) / VOLUMEN_AIRE
        
        # Reducción adicional necesaria
        reduccion_adicional = max(0, reduccion_necesaria - reduccion_ya_lograda)
        
        # Árboles nuevos necesarios este año
        if reduccion_adicional > 0:
            nuevos_arboles = int(np.ceil((reduccion_adicional * VOLUMEN_AIRE) / (reduccion_diaria_por_arbol * 1000)))
        else:
            nuevos_arboles = 0
            
        arboles_necesarios_por_año[año] = nuevos_arboles
        arboles_acumulados[año] = arboles_acumulados_previos + (nuevos_arboles * TASA_SUPERVIVENCIA)
        
        # Calcular PM10 proyectado con estos árboles
        reduccion_total = (reduccion_diaria_por_arbol * arboles_acumulados[año] * 1000) / VOLUMEN_AIRE
        pm10_proyectado[año] = pm10_base_año - reduccion_total
    
    return arboles_necesarios_por_año, arboles_acumulados, pm10_proyectado, prediccion_base

def simular_escenarios_multiple(df_pred, start_year=2025, escenarios_arboles=[120000, 200000, 300000, 400000, 500000]):
    """
    Simula múltiples escenarios con diferentes cantidades de árboles plantados anualmente
    """
    prediccion_base = modelo.predict(df_pred)
    resultados_escenarios = {}
    
    # Calcular reducción por árbol (5217.4 g/año)
    reduccion_diaria_por_arbol = 5217.4 / 365
    
    años_unicos = df_pred['AÑO'].unique()
    
    for arboles_anuales in escenarios_arboles:
        prediccion_escenario = prediccion_base.copy()
        arboles_acumulados = {}
        
        for i, año in enumerate(años_unicos):
            # Árboles acumulados considerando supervivencia
            n_arboles = int(arboles_anuales * TASA_SUPERVIVENCIA * (i + 1))
            arboles_acumulados[año] = n_arboles
            
            reduccion_diaria = reduccion_diaria_por_arbol * n_arboles
            reduccion_concentracion = (reduccion_diaria * 1000) / VOLUMEN_AIRE
            
            mask_año = df_pred['AÑO'] == año
            prediccion_escenario[mask_año] = np.maximum(0, 
                prediccion_escenario[mask_año] - reduccion_concentracion)
        
        resultados_escenarios[arboles_anuales] = (prediccion_escenario, arboles_acumulados)
    
    return prediccion_base, resultados_escenarios

def categorizar_calidad_aire(valor):
    if valor <= 54:
        return 'Buena'
    elif valor <= 154:
        return 'Moderada'
    elif valor <= 254:
        return 'Insalubre para grupos sensibles'
    else:
        return 'Insalubre'


In [17]:
# Crear datos para predicción
df_prediccion = crear_datos_prediccion(2025, 2035)

# Calcular árboles necesarios para alcanzar el objetivo de PM10
arboles_necesarios, arboles_acumulados, pm10_proyectado, pred_base = calcular_arboles_necesarios(
    df_prediccion, target_pm10=(TARGET_PM10_MIN + TARGET_PM10_MAX) / 2
)

# También simular escenarios con cantidades fijas para comparación
escenarios_arboles = [120000, 200000, 300000, 400000, 500000]
_, resultados_escenarios = simular_escenarios_multiple(df_prediccion, escenarios_arboles=escenarios_arboles)

# Crear visualización para árboles necesarios
fig1 = make_subplots(rows=2, cols=1,
                    subplot_titles=('Evolución proyectada de PM10 hacia nivel objetivo',
                                  'Árboles necesarios por año para alcanzar objetivo'))

# Añadir datos históricos
fig1.add_trace(
    go.Scatter(x=list(historico_pm10.keys()),
               y=list(historico_pm10.values()),
               name='Histórico PM10',
               line=dict(color='gray', dash='dash')),
    row=1, col=1
)

# Añadir línea de predicción base
años_proyeccion = list(pm10_proyectado.keys())
fig1.add_trace(
    go.Scatter(x=años_proyeccion,
               y=[np.mean(pred_base[df_prediccion['AÑO'] == año]) for año in años_proyeccion],
               name='Proyección sin árboles',
               line=dict(color='red')),
    row=1, col=1
)

# Añadir línea de PM10 con árboles necesarios
fig1.add_trace(
    go.Scatter(x=años_proyeccion,
               y=list(pm10_proyectado.values()),
               name='PM10 con árboles necesarios',
               line=dict(color='green')),
    row=1, col=1
)

# Área objetivo de PM10
fig1.add_trace(
    go.Scatter(
        x=[años_proyeccion[0], años_proyeccion[-1], años_proyeccion[-1], años_proyeccion[0]],
        y=[TARGET_PM10_MIN, TARGET_PM10_MIN, TARGET_PM10_MAX, TARGET_PM10_MAX],
        fill='toself',
        fillcolor='rgba(0,255,0,0.2)',
        line=dict(color='rgba(0,255,0,0.5)'),
        name=f'Rango objetivo ({TARGET_PM10_MIN}-{TARGET_PM10_MAX} µg/m³)'
    ),
    row=1, col=1
)

# Añadir barras de árboles necesarios por año
fig1.add_trace(
    go.Bar(x=años_proyeccion,
           y=list(arboles_necesarios.values()),
           name='Árboles necesarios por año',
           marker_color='blue'),
    row=2, col=1
)

# Añadir línea de referencia para el promedio actual
fig1.add_trace(
    go.Scatter(x=años_proyeccion,
               y=[ARBOLES_PROMEDIO] * len(años_proyeccion),
               name=f'Promedio actual ({int(ARBOLES_PROMEDIO):,} árboles/año)',
               line=dict(color='orange', dash='dash')),
    row=2, col=1
)

# Actualizar diseño
fig1.update_layout(
    title='Simulación de árboles necesarios para alcanzar nivel objetivo de PM10',
    height=800,
    showlegend=True
)

fig1.show()

In [18]:
# Crear una segunda visualización para comparar diferentes escenarios
fig2 = go.Figure()

# Añadir datos históricos
fig2.add_trace(
    go.Scatter(x=list(historico_pm10.keys()),
               y=list(historico_pm10.values()),
               name='Histórico PM10',
               line=dict(color='gray', dash='dash'))
)

# Añadir línea de predicción base
fig2.add_trace(
    go.Scatter(x=años_proyeccion,
               y=[np.mean(pred_base[df_prediccion['AÑO'] == año]) for año in años_proyeccion],
               name='Proyección sin árboles',
               line=dict(color='red'))
)

# Añadir escenarios
colores = ['blue', 'green', 'purple', 'orange', 'teal']
for i, arboles in enumerate(escenarios_arboles):
    prediccion_escenario, _ = resultados_escenarios[arboles]
    pm10_por_año = [np.mean(prediccion_escenario[df_prediccion['AÑO'] == año]) for año in años_proyeccion]
    
    fig2.add_trace(
        go.Scatter(x=años_proyeccion,
                   y=pm10_por_año,
                   name=f'{arboles:,} árboles/año',
                   line=dict(color=colores[i % len(colores)]))
    )

# Añadir línea para escenario de árboles necesarios calculados
fig2.add_trace(
    go.Scatter(x=años_proyeccion,
               y=list(pm10_proyectado.values()),
               name='Árboles necesarios calculados',
               line=dict(color='black', width=2))
)

# Área objetivo de PM10
fig2.add_trace(
    go.Scatter(
        x=[años_proyeccion[0], años_proyeccion[-1], años_proyeccion[-1], años_proyeccion[0]],
        y=[TARGET_PM10_MIN, TARGET_PM10_MIN, TARGET_PM10_MAX, TARGET_PM10_MAX],
        fill='toself',
        fillcolor='rgba(0,255,0,0.2)',
        line=dict(color='rgba(0,255,0,0.5)'),
        name=f'Rango objetivo ({TARGET_PM10_MIN}-{TARGET_PM10_MAX} µg/m³)'
    )
)

# Actualizar diseño
fig2.update_layout(
    title='Comparación de escenarios de reforestación y niveles de PM10',
    xaxis_title='Año',
    yaxis_title='PM10 (µg/m³)',
    height=600,
    showlegend=True
)

fig2.show()

In [19]:
# Imprimir estadísticas
print("\nEstadísticas de la simulación:")
print(f"Objetivo de PM10: {TARGET_PM10_MIN}-{TARGET_PM10_MAX} µg/m³")
print(f"Promedio actual de árboles plantados por año (SERPAR): {int(ARBOLES_PROMEDIO):,}")
print(f"Tasa de supervivencia considerada: {TASA_SUPERVIVENCIA*100}%")

print("\nÁrboles necesarios para alcanzar el objetivo:")
total_arboles = sum(arboles_necesarios.values())
print(f"Total de árboles a plantar (2025-2035): {total_arboles:,}")
print(f"Promedio anual necesario: {int(total_arboles/len(arboles_necesarios)):,}")

for año, arboles in arboles_necesarios.items():
    print(f"\nAño {año}:")
    print(f"Árboles necesarios: {arboles:,}")
    print(f"Árboles acumulados (con supervivencia): {int(arboles_acumulados[año]):,}")
    base_pm10 = np.mean(pred_base[df_prediccion['AÑO'] == año])
    print(f"PM10 sin árboles: {base_pm10:.2f} µg/m³")
    print(f"PM10 proyectado: {pm10_proyectado[año]:.2f} µg/m³")
    reduccion = base_pm10 - pm10_proyectado[año]
    print(f"Reducción: {reduccion:.2f} µg/m³")

print("\nComparación de escenarios:")
for arboles in escenarios_arboles:
    prediccion_escenario, _ = resultados_escenarios[arboles]
    pm10_2035 = np.mean(prediccion_escenario[df_prediccion['AÑO'] == 2035])
    print(f"{arboles:,} árboles/año -> PM10 en 2035: {pm10_2035:.2f} µg/m³")


Estadísticas de la simulación:
Objetivo de PM10: 27-40.5 µg/m³
Promedio actual de árboles plantados por año (SERPAR): 119,242
Tasa de supervivencia considerada: 70.0%

Árboles necesarios para alcanzar el objetivo:
Total de árboles a plantar (2025-2035): 40,110,741
Promedio anual necesario: 3,646,431

Año 2025:
Árboles necesarios: 27,422,076
Árboles acumulados (con supervivencia): 19,195,453
PM10 sin árboles: 47.65 µg/m³
PM10 proyectado: 37.92 µg/m³
Reducción: 9.73 µg/m³

Año 2026:
Árboles necesarios: 7,714,663
Árboles acumulados (con supervivencia): 24,595,717
PM10 sin árboles: 47.40 µg/m³
PM10 proyectado: 34.92 µg/m³
Reducción: 12.47 µg/m³

Año 2027:
Árboles necesarios: 2,191,044
Árboles acumulados (con supervivencia): 26,129,448
PM10 sin árboles: 47.33 µg/m³
PM10 proyectado: 34.08 µg/m³
Reducción: 13.25 µg/m³

Año 2028:
Árboles necesarios: 2,481,042
Árboles acumulados (con supervivencia): 27,866,177
PM10 sin árboles: 48.26 µg/m³
PM10 proyectado: 34.13 µg/m³
Reducción: 14.13 µg/m³

A